In [16]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy
import json
%matplotlib inline

np.set_printoptions(threshold=np.nan)

In [17]:
#Retrieve Parameters
params = pd.read_csv("ML_param.csv")
#params.values
#params.iterrows()
del params['coord_dec'] #Irrelevant (Hopefully) and poorly formatted for ML
del params['coord_ra']
del params['id']
del params['parent']


In [18]:
#Retrieve Training Set Classifications
csv_path = "../classifications/lsst_run_one.csv"
clsfn = pd.read_csv(csv_path)
subset = clsfn.loc[clsfn['workflow_name'] == "Difference Imaging Classifier"]
np.shape(subset)
im_class = []
for item, row in subset.iterrows():
    s_data = json.loads(row.subject_data) #Subject Data
    s_data = s_data.get(s_data.keys()[0])
    im=s_data.get(s_data.keys()[0])[47:-4]
    a_data = json.loads(row.annotations)[0] #Annotations
    classification = a_data['value']
    im_class.append([int(im),classification])
im_class = sorted(im_class)
#im_class

In [19]:
#Generate Data Frame that holds classifications and respective measured quantities
#This is done as a dataframe is a more cohesive object for data analysis
d = {}
for key in params.keys():
    d[key] = []
d["Classification"]=[]
im_data = params.set_index('image').T.to_dict()
for im in im_class:
    d["Classification"].append(im[1])
    d["image"].append(im[0])
    dat_dict = im_data.get(im[0])
    for key in dat_dict:
        d[key].append(dat_dict.get(key))
df = pd.DataFrame(d)    

In [20]:
#Read in the features in the data frame, filter for columns relevant for ML
features = df.columns.tolist()

features = [c for c in features if c not in ["image", "Classification"]]
target = "Classification"  #Predict on Classifications

#Cut 'useless' features and problematic features
for feature in features:
    if df[feature].isnull().all():
        del df[feature]   
    elif np.mean(df[feature]) == np.inf:
        del df[feature]
    elif "flag" in feature: #Flags don't contribute to ML based on initial testing
        del df[feature]
        
features = df.columns.tolist()

features = [c for c in features if c not in ["image", "Classification"]]

In [21]:
imp = Imputer(missing_values='NaN', strategy='median', axis=0, verbose = 1)
imp.fit(df[features])
features_imp = imp.transform(df[features])

In [22]:
#Random Forest
RFC =  RandomForestClassifier()
RFC = RFC.fit(features_imp,df[target])

In [26]:
feature_importance = zip(RFC.feature_importances_, features)
sorted(feature_importance, reverse=True)

[(0.12701897296072046, 'base_CircularApertureFlux_17_0_fluxSigma'),
 (0.11834613397314279, 'base_CircularApertureFlux_6_0_fluxSigma'),
 (0.062087797283638967, 'base_CircularApertureFlux_3_0_fluxSigma'),
 (0.056535539383436792, 'base_CircularApertureFlux_12_0_fluxSigma'),
 (0.055544874792905083, 'ip_diffim_NaiveDipoleFlux_pos_fluxSigma'),
 (0.054642232490412411, 'base_CircularApertureFlux_3_0_flux'),
 (0.037724480245909797, 'base_PeakLikelihoodFlux_flux'),
 (0.036225573098795712, 'base_CircularApertureFlux_4_5_fluxSigma'),
 (0.032024622981066272, 'base_PsfFlux_fluxSigma'),
 (0.031394855245351391, 'base_PeakLikelihoodFlux_fluxSigma'),
 (0.02716943217282063, 'base_CircularApertureFlux_6_0_flux'),
 (0.020736123968147922, 'base_CircularApertureFlux_35_0_flux'),
 (0.020168995751067828, 'ip_diffim_NaiveDipoleFlux_pos_flux'),
 (0.019714614461649013, 'base_SdssShape_yy'),
 (0.01835438418251796, 'base_PeakCentroid_x'),
 (0.017053205721968646, 'base_GaussianFlux_flux'),
 (0.016036722460009922, 'b

In [27]:
df.corr()

base_CircularApertureFlux_12_0_flux  \
base_CircularApertureFlux_12_0_flux                                  1.000000   
base_CircularApertureFlux_12_0_fluxSigma                             0.036540   
base_CircularApertureFlux_17_0_flux                                  0.995484   
base_CircularApertureFlux_17_0_fluxSigma                             0.036532   
base_CircularApertureFlux_25_0_flux                                  0.967055   
base_CircularApertureFlux_35_0_flux                                  0.802827   
base_CircularApertureFlux_3_0_flux                                   0.984789   
base_CircularApertureFlux_3_0_fluxSigma                              0.033081   
base_CircularApertureFlux_4_5_flux                                   0.986195   
base_CircularApertureFlux_4_5_fluxSigma                              0.036098   
base_CircularApertureFlux_50_0_flux                                  0.514077   
base_CircularApertureFlux_6_0_flux                                   0.986674   
base_CircularApertureFlux_6_0_fluxSigma                              0.036391   
base_CircularApertureFlux_70_0_flux                                  0.200244   
base_CircularApertureFlux_9_0_flux                                   0.994368   
base_GaussianCentroid_x                                             -0.136409   
base_GaussianCentroid_y                                              0.182470   
base_GaussianFlux_flux                                               0.946696   
base_GaussianFlux_fluxSigma                                         -0.072619   
base_NaiveCentroid_x                                                -0.136455   
base_NaiveCentroid_y                                                 0.182444   
base_PeakCentroid_x                                                 -0.136319   
base_PeakCentroid_y                                                  0.182484   
base_PeakLikelihoodFlux_flux                                         0.912738   
base_PeakLikelihoodFlux_fluxSigma                                   -0.188407   
base_PsfFlux_flux                                                    0.981272   
base_PsfFlux_fluxSigma                                               0.036137   
base_SdssCentroid_x                                                 -0.136368   
base_SdssCentroid_xSigma                                            -0.055686   
base_SdssCentroid_y                                                  0.182418   
...                                                                       ...   
ip_diffim_DipoleFit_neg_fluxSigma                                    0.090539   
ip_diffim_DipoleFit_orientation                                     -0.072059   
ip_diffim_DipoleFit_pos_centroid_x                                   0.124519   
ip_diffim_DipoleFit_pos_centroid_y                                  -0.446234   
ip_diffim_DipoleFit_pos_flux                                         0.123500   
ip_diffim_DipoleFit_pos_fluxSigma                                    0.090539   
ip_diffim_DipoleFit_separation                                      -0.003462   
ip_diffim_DipoleFit_signalToNoise                                    0.090539   
ip_diffim_NaiveDipoleCentroid_neg_x                                  0.051101   
ip_diffim_NaiveDipoleCentroid_neg_y                                 -0.113827   
ip_diffim_NaiveDipoleCentroid_pos_x                                 -0.095190   
ip_diffim_NaiveDipoleCentroid_pos_y                                  0.264945   
ip_diffim_NaiveDipoleCentroid_x                                     -0.136346   
ip_diffim_NaiveDipoleCentroid_y                                      0.182426   
ip_diffim_NaiveDipoleFlux_neg_flux                                   0.076727   
ip_diffim_NaiveDipoleFlux_nneg                                       0.541844   
ip_diffim_NaiveDipoleFlux_npos                                       0.872379   
ip_diffim_NaiveDipoleFlux_pos_flux                                   0.97

In [28]:
#Split into Training Set and Testing Set

train = df.sample(frac=0.8, random_state=1)
test = df.loc[~df.index.isin(train.index)]

imp_tt = Imputer(missing_values='NaN', strategy='median', axis=0, verbose = 1)
imp_tt.fit(train[features])
features_imp_tt = imp_tt.transform(train[features])

#Random Forest Training/Test Split
RFC_tt =  RandomForestClassifier()
RFC_tt = RFC.fit(features_imp_tt,train[target])

In [29]:
imp_test = Imputer(missing_values='NaN', strategy='median', axis=0, verbose = 1)
imp_test.fit(test[features])
features_imp_test = imp_test.transform(test[features])


predictions = RFC_tt.predict(features_imp_test)

In [30]:
for index in range(len(predictions)):
    print predictions[index],",",test["Classification"].tolist()[index]

In [31]:
for index in range(len(predictions)):
    if predictions[index] != test["Classification"].tolist()[index]:
        print predictions[index],",",test["Classification"].tolist()[index],",",test["image"].tolist()[index]

In [32]:
from collections import Counter

In [48]:
pred_mapping = {}
for index in range(len(predictions)):
    pred_name = predictions[index]
    class_name = test["Classification"].tolist()[index]
    if pred_name in pred_mapping:
        pred_mapping[pred_name].append(class_name)
    else:
        pred_mapping[pred_name] = [class_name]
        
for pred_name, pred_value in pred_mapping.iteritems():
    print pred_name, Counter(pred_value)

In [49]:
agree=0
disagree=0
for index in range(len(predictions)):
    if predictions[index] == test["Classification"].tolist()[index]:
        agree+=1
    else:
        disagree+=1

print "Agree: "+ str(agree), "Disagree: "+str(disagree)
        

In [41]:
(75+82.)/(247+75)

0.48757763975155277

In [46]:
percent=float(agree)/len(predictions)
print percent

In [ ]:
#Bring up images of disagreement